In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate



In [ ]:
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
#ensure that TensorFlow can access local GPUs

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)


Load Parquet Data

In [ ]:
small_set_data = pd.read_parquet("data/Zzzs_train.parquet")

In [ ]:
data_series = {}
for series in set(small_set_data["series_id"]):
    data_series[series] = small_set_data[small_set_data["series_id"] == series]

In [ ]:
dd = data_series['d150801f3145']

In [ ]:
dd = data_series['67f5fc60e494']

In [ ]:
dd.reset_index(inplace= True)

In [ ]:
for series in set(small_set_data["series_id"]):
    print(f'{series} head:---------------')
    print(data_series[series].head())
    print(data_series[series].tail())
    
    
    

In [ ]:
def create_non_overlapping_day_windows(df,feature):
    # Assuming your DataFrame is named df and the timestamp column is 'timestamp'
    # Make a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()
    
    # Reset the index to get the timestamp as a column again
    df_copy.reset_index(inplace=True)
    print(df_copy.head())

    # Assuming your DataFrame has a 'timestamp' column
    df_copy['timestamp'] = pd.to_datetime(df_copy['timestamp'])
    
    #df_copy['timestamp'] = pd.to_datetime(df_copy['timestamp'], errors='raise')

    df_copy['date'] = df_copy['timestamp'].dt.date
    

    # Set the timestamp column as the index
    df_copy.set_index('timestamp', inplace=True)

    # Resample the data to create sections by day and fill missing values with 0
    #df_resampled = df_copy.resample('D').fillna(method='nearest')
    df_resampled = df_copy#.fillna(method='ffill')
    # Reset the index to get the timestamp as a column again
    df_resampled.reset_index(inplace=True)
    
    
    # Find the minimum and maximum dates
    min_date = df_copy['date'].min()
    max_date = df_copy['date'].max()
    
    df_resampled = df_copy.fillna(method='ffill')
    
    df_resampled_deended =  df_resampled[(df_resampled['date'] >min_date) & (df_resampled['date']<max_date)]
    print('-------------------------------------------------------------')
    print('filtered', df_resampled_deended)
    # Create a list to store individual DataFrames for each day
    daily_dataframes = []

    # Iterate through each day in the resampled DataFrame
    for _, day_data in df_resampled_deended.groupby("date"):
        # Append the DataFrame for the current day to the list
        print(day_data)
        print(day_data.shape)
        daily_dataframes.append(day_data)
    
    column_names = ["local step " + str(i) for i in range(len(daily_dataframes[0]))]
    column_names = ["series_id", "global first step num", "global first timestamp"]+column_names
    column_names = column_names + ["local label " + str(i) for i in range(len(daily_dataframes[0]))]
    windowed_data = []
    #dummy_counter = 10
    #num_section = len(data) // window_size
    for day in daily_dataframes:
        
        current_row = list(day.iloc[0,0:3]) + list(day[feature]) + list(day["awake"])
        '''
        #print(i, sum(data["enmo"][i:i+window_size]))
        if sum(data["awake"].iloc[i:i+window_size]) >= window_size - buffer: #Therefore during a time window, if the majority of the timesteps are
            #awake then the window is defined as awake; it does not matter if the window started with awake as 0, or 1.
            state = [1,0,0,0]
        elif sum(data["awake"].iloc[i:i+window_size]) <= buffer:   
            state = [0,0,0,1]
        elif data["awake"].iloc[i] == 1:   
            state = [0,0,1,0]
        elif data["awake"].iloc[i] == 0:   
            state = [0,1,0,0] 
        current_row += state
        '''
        #print(len(current_row))
        #print(current_row)
        #print(type(windowed_data))
        windowed_data.append(current_row)
        '''
        if (i*100) // (len(data) - window_size + 1) % 10 == 0 and (i  * 100 )// (len(data) - window_size + 1) != 0 and (i*100) // (len(data) - window_size + 1) >= dummy_counter:
            print(f"series {data.iloc[i,0]}  completed: {(i*100) // (len(data) - window_size + 1)} %")
            dummy_counter += 10
        '''
    windowed_data = pd.DataFrame(windowed_data, columns=column_names) #windowed_data.append(pd.Series(current_row, index=windowed_data.columns), ignore_index=True)
        
    return windowed_data
    
        
        
        
    

In [ ]:
# Function to convert timestamp string to sinusoidal wave representation
def convert_to_sine_wave(timestamp_str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
    time_in_hours = timestamp_dt.hour + timestamp_dt.minute / 60  # Convert minute to fraction of an hour
    return np.sin(2 * np.pi * time_in_hours / 24)  # 24 hours is the period


In [ ]:
# Function to convert timestamp string to numerical value
def convert_to_numerical(timestamp_str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
    return timestamp_dt.hour * 100 + timestamp_dt.minute

In [ ]:
def convert_to_date(timestamp_str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
    return timestamp_dt.date

In [ ]:
def convert_to_numeric_date(timestamp_str):
    timestamp_dt = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
    numeric_date = int(timestamp_dt.strftime('%Y%m%d'))
    return numeric_date

In [ ]:
# Iterate through the dictionary and modify each DataFrame
from datetime import datetime
for key, df in data_series.items():
    df_copy = df.copy()
    # Apply the custom function to the specified feature and create a new column
    df_copy['sinusoidal_time'] = df_copy['timestamp'].apply(convert_to_sine_wave)
    df_copy['numerical_time'] = df_copy['timestamp'].apply(convert_to_numerical)
    data_series[key] = df_copy

In [ ]:
discard_series = ['67f5fc60e494','a9a2f7fac455','d5e47b94477e']

for series in discard_series:
    
    data_series.pop(series, None)

In [ ]:
for key, df in data_series.items():
    df_copy = df.copy()
    # Apply the custom function to the specified feature and create a new column
    #df_copy['sinusoidal_time'] = df_copy['timestamp'].apply(convert_to_sine_wave)
    df_copy['date'] = df_copy['timestamp'].apply(convert_to_date)
    data_series[key] = df_copy

In [ ]:
for key, df in data_series.items():
    df_copy = df.copy()
    # Apply the custom function to the specified feature and create a new column
    #df_copy['sinusoidal_time'] = df_copy['timestamp'].apply(convert_to_sine_wave)
    df_copy['numeric_date'] = df_copy['timestamp'].apply(convert_to_numeric_date)
    data_series[key] = df_copy

In [ ]:
for _, df in data_series.items():
    print(df.head())
    df_copy = df.copy()
    invalid_rows = df_copy[df_copy['timestamp'] == "NaT"]

In [ ]:

#window_size = 50
#buffer_percentage = 10
variable_list = ['anglez','enmo','sinusoidal_time','numerical_time']
windowed_dataset = {}
indices_with_errors = []
for variable in variable_list:
    windowed_dataset[variable] = pd.DataFrame()

    for key,series in data_series.items():#[2:]:
        #print(data_series[series].head())
        try:
            windowed_series = create_non_overlapping_day_windows(df = series,feature = variable)   
            windowed_dataset[variable] = pd.concat([windowed_dataset[variable], windowed_series], ignore_index=True)
        except Exception as e:
            print(f"Error at index {key}: {e}")
            indices_with_errors.append(key)
            continue

In [166]:
print(len(set(indices_with_errors)))

32


In [167]:
windowed_dataset['anglez'].shape

(562, 34563)

In [171]:
windowed_dataset['anglez'].head()

,series_id,global first step num,global first timestamp,local step 0,local step 1,local step 2,local step 3,local step 4,local step 5,local step 6,...,local label 17270,local label 17271,local label 17272,local label 17273,local label 17274,local label 17275,local label 17276,local label 17277,local label 17278,local label 17279
0,2017-08-29 00:00:00-04:00,11776680,d150801f3145,-67.953598,-67.910500,-67.851196,-67.919502,-67.936699,-67.897102,-67.839104,...,1,1,1,1,1,1,1,1,1,1
1,2017-08-30 00:00:00-04:00,11793960,d150801f3145,-14.307700,-14.307700,-14.307700,-14.307700,-14.305400,-14.296400,-14.299800,...,0,0,0,0,0,0,0,0,0,0
2,2017-08-31 00:00:00-04:00,11811240,d150801f3145,-43.974800,-43.975899,-43.974800,-43.974800,-43.974800,-43.974800,-43.974800,...,0,0,0,0,0,0,0,0,0,0
3,2017-09-01 00:00:00-04:00,11828520,d150801f3145,5.180900,4.994500,5.184000,5.222100,5.129400,5.068500,5.195000,...,0,0,0,0,0,0,0,0,0,0
4,2017-09-02 00:00:00-04:00,11845800,d150801f3145,-58.329899,-58.280998,-58.234501,-58.244301,-58.234501,-58.237000,-58.234501,...,0,0,0,0,0,0,0,0,0,0


In [169]:
print(set(windowed_dataset['anglez']['series_id']))

{Timestamp('2017-09-30 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2017-09-14 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-04-30 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-04-14 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-09-18 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-05-09 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2017-12-13 00:00:00-0500', tz='pytz.FixedOffset(-300)'), Timestamp('2019-06-30 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-02-20 00:00:00-0500', tz='pytz.FixedOffset(-300)'), Timestamp('2017-09-13 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-05-04 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-09-17 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-09-01 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-04-22 00:00:00-0400', tz='pytz.FixedOffset(-240)'), Timestamp('2018-12-28 00:00:00-0500', tz='pytz.

In [173]:
#Loading Data

# Load data from the first CSV file
#file_path1 = 'data/windowed_sequences/50_small_set_enmo_windowed.csv'

#file_path2 = 'data/windowed_sequences/50_small_set_anglez_windowed.csv'

#file1_data = pd.read_csv(file_path1)#, dtype=np.float32)

# Set the first column as the index
#file1_data.set_index(file1_data.columns[0], inplace=True)

# Convert the DataFrame to a NumPy array
#drop_columns = ["series_id", "global first step num", "global first timestamp"] + ["local label " + str(i) for i in range(int(24 * 60 * 60 / 5))]
drop_columns = ["global first step num", "global first timestamp"] + ["local label " + str(i) for i in range(int(24 * 60 * 60 / 5))]

lable_columns = ["local label " + str(i) for i in range(int(24 * 60 * 60 / 5))]

variable_data = []
for variable in variable_list:
    df_copy = windowed_dataset[variable].copy()
    data_array = df_copy.drop(columns=drop_columns).values
    variable_data.append(data_array)
    
#file1_array = file1_data.drop(columns=drop_columns).values

# Load data from the second CSV file
#file2_data = pd.read_csv(file_path2)

#file2_array = file2_data.drop(columns=drop_columns).values
df_copy = windowed_dataset[variable_list[0]].copy()
data_identity_storage = df_copy[["series_id", "global first step num", "global first timestamp"]]
df_Y = df_copy[lable_columns]
Y_all = df_Y.values
# Check shapes
for i in range(len(variable_data)-1):
    assert variable_data[i].shape == variable_data[i+1].shape, "Arrays should have the same shape"


# Create 3D array
three_d_array = np.stack(variable_data, axis=2)


KeyError: "['global first step num', 'global first timestamp', 'local label 0', 'local label 1', 'local label 2', 'local label 3', 'local label 4', 'local label 5', 'local label 6', 'local label 7', 'local label 8', 'local label 9', 'local label 10', 'local label 11', 'local label 12', 'local label 13', 'local label 14', 'local label 15', 'local label 16', 'local label 17', 'local label 18', 'local label 19', 'local label 20', 'local label 21', 'local label 22', 'local label 23', 'local label 24', 'local label 25', 'local label 26', 'local label 27', 'local label 28', 'local label 29', 'local label 30', 'local label 31', 'local label 32', 'local label 33', 'local label 34', 'local label 35', 'local label 36', 'local label 37', 'local label 38', 'local label 39', 'local label 40', 'local label 41', 'local label 42', 'local label 43', 'local label 44', 'local label 45', 'local label 46', 'local label 47', 'local label 48', 'local label 49', 'local label 50', 'local label 51', 'local labe

In [ ]:
# Extract indices from the original 3D array
indices_list = list(range(three_d_array.shape[0]))

In [ ]:
print(three_d_array.shape)

In [ ]:


# Split the data and indices into training and validation sets
indices_train, indices_val, X_train, X_val, Y_train, Y_val = train_test_split(
    indices_list, three_d_array, Y_all, test_size=0.2, random_state=42, stratify=Y_all
)

# Check the shapes of the training and validation sets
print("indices_train shape:", len(indices_train))
print("X_train shape:", X_train.shape)
print("y_train shape:", Y_train.shape)
print("indices_val shape:", len(indices_val))
print("X_val shape:", X_val.shape)
print("y_val shape:", Y_val.shape)

In [ ]:
'''
# Adjust the size of the subset based on your computational resources
subset_percentage = 10
subset_size = X_train.shape[0] // 100 * subset_percentage

# Split the data into a subset and the rest
subset_X, remaining_X, subset_Y, remaining_Y = train_test_split(X_train, Y_train, test_size=subset_size, random_state=42, stratify=Y_train)

# Confirm the shapes of the subsets
print("Subset X shape:", subset_X.shape)
print("Remaining X shape:", remaining_X.shape)
print("Subset y shape:", subset_Y.shape)
print("Remaining y shape:", remaining_Y.shape)

In [ ]:
#for tuning model configuration
dataset_for_training = X_train
labels_for_training = Y_train
num_classes = labels_for_training.shape[1]

# Assuming you have sequences (X) and labels (y) for training
# X should be a 3D tensor with shape (num_samples, sequence_length, num_features)
# y should be a 2D tensor with shape (num_samples, num_classes)

sequence_length = dataset_for_training.shape[1]
num_features = dataset_for_training.shape[2]

# Define the LSTM model
input_sequence = Input(shape=(sequence_length, num_features))
#lstm_out = LSTM(units=64, return_sequences=True)(input_sequence)
lstm_out = LSTM(units=128, return_sequences=False, recurrent_initializer='glorot_uniform', implementation=2)(input_sequence)


# Define the attention mechanism
attention = Attention()([lstm_out, lstm_out])

# Concatenate the LSTM output and attention output
merged = Concatenate(axis=-1)([lstm_out, attention])

# Fully connected layer for final prediction
output = Dense(num_classes, activation='softmax')(merged)

# Create the hybrid model
model = Model(inputs=input_sequence, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(labels_for_training, axis=1)), y=np.argmax(labels_for_training, axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/4_series_4_var_LSTM128_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)

# Train the model with class weights, early stopping, and TensorBoard
#model.fit(subset_X, subset_Y, epochs=10, batch_size=32, validation_data=(X_val, Y_val), class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard])
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_for_training, labels_for_training)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32)

model.fit(train_dataset, epochs=20, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, Y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Trying 100 sequences

In [ ]:

window_size = 100
buffer_percentage = 10
variable_list = ['anglez','enmo','sinusoidal_time','sinusoidal_time']
windowed_dataset = {}
for variable in variable_list:
    windowed_dataset[variable] = pd.DataFrame()

    for series in list(set(small_set_data["series_id"]))[:4]:
     
        windowed_series = create_sliding_windows_W_buffer(data_series[series],window_size = window_size,buffer=(window_size * buffer_percentage / 100),feature = variable)   
        windowed_dataset[variable] = pd.concat([windowed_dataset[variable], windowed_series], ignore_index=True)

In [ ]:
#LSTM = 128

#for tuning model configuration
dataset_for_training = X_train
labels_for_training = Y_train
num_classes = labels_for_training.shape[1]

# Assuming you have sequences (X) and labels (y) for training
# X should be a 3D tensor with shape (num_samples, sequence_length, num_features)
# y should be a 2D tensor with shape (num_samples, num_classes)

sequence_length = dataset_for_training.shape[1]
num_features = dataset_for_training.shape[2]

# Define the LSTM model
input_sequence = Input(shape=(sequence_length, num_features))
#lstm_out = LSTM(units=64, return_sequences=True)(input_sequence)
lstm_out = LSTM(units=128, return_sequences=False, recurrent_initializer='glorot_uniform', implementation=2)(input_sequence)


# Define the attention mechanism
attention = Attention()([lstm_out, lstm_out])

# Concatenate the LSTM output and attention output
merged = Concatenate(axis=-1)([lstm_out, attention])

# Fully connected layer for final prediction
output = Dense(num_classes, activation='softmax')(merged)

# Create the hybrid model
model = Model(inputs=input_sequence, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(labels_for_training, axis=1)), y=np.argmax(labels_for_training, axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/model_one_series_LSTM128_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)

# Train the model with class weights, early stopping, and TensorBoard
#model.fit(subset_X, subset_Y, epochs=10, batch_size=32, validation_data=(X_val, Y_val), class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard])
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_for_training, labels_for_training)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32)

model.fit(train_dataset, epochs=20, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, Y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


In [ ]:
loaded_model = load_model('model/model_one_series_LSTM128_W_attention.h5')

#for tuning model configuration
dataset_for_training = X_train
labels_for_training = Y_train

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(labels_for_training, axis=1)), y=np.argmax(labels_for_training, axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/model_one_series_LSTM_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)

# Train the model with class weights, early stopping, and TensorBoard
#model.fit(subset_X, subset_Y, epochs=10, batch_size=32, validation_data=(X_val, Y_val), class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard])
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_for_training, labels_for_training)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32)

loaded_model.fit(train_dataset, epochs=20, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])


In [ ]:
loaded_model = load_model('model/model_one_series_LSTM128_W_attention.h5')

#for tuning model configuration
dataset_for_training = X_train
labels_for_training = Y_train

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(labels_for_training, axis=1)), y=np.argmax(labels_for_training, axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/model_one_series_LSTM_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)

# Train the model with class weights, early stopping, and TensorBoard
#model.fit(subset_X, subset_Y, epochs=10, batch_size=32, validation_data=(X_val, Y_val), class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard])
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_for_training, labels_for_training)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32)

loaded_model.fit(train_dataset, epochs=20, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])


In [ ]:
#LSTM = 32

#for tuning model configuration
dataset_for_training = X_train
labels_for_training = Y_train
num_classes = labels_for_training.shape[1]

# Assuming you have sequences (X) and labels (y) for training
# X should be a 3D tensor with shape (num_samples, sequence_length, num_features)
# y should be a 2D tensor with shape (num_samples, num_classes)

sequence_length = dataset_for_training.shape[1]
num_features = dataset_for_training.shape[2]

# Define the LSTM model
input_sequence = Input(shape=(sequence_length, num_features))
#lstm_out = LSTM(units=64, return_sequences=True)(input_sequence)
lstm_out = LSTM(units=32, return_sequences=False, recurrent_initializer='glorot_uniform', implementation=2)(input_sequence)


# Define the attention mechanism
attention = Attention()([lstm_out, lstm_out])

# Concatenate the LSTM output and attention output
merged = Concatenate(axis=-1)([lstm_out, attention])

# Fully connected layer for final prediction
output = Dense(num_classes, activation='softmax')(merged)

# Create the hybrid model
model = Model(inputs=input_sequence, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(labels_for_training, axis=1)), y=np.argmax(labels_for_training, axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/model_one_series_LSTM32_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)

# Train the model with class weights, early stopping, and TensorBoard
#model.fit(subset_X, subset_Y, epochs=10, batch_size=32, validation_data=(X_val, Y_val), class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard])
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_for_training, labels_for_training)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32)

model.fit(train_dataset, epochs=30, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, Y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


In [ ]:
#LSTM = 256

#for tuning model configuration
dataset_for_training = X_train
labels_for_training = Y_train
num_classes = labels_for_training.shape[1]

# Assuming you have sequences (X) and labels (y) for training
# X should be a 3D tensor with shape (num_samples, sequence_length, num_features)
# y should be a 2D tensor with shape (num_samples, num_classes)

sequence_length = dataset_for_training.shape[1]
num_features = dataset_for_training.shape[2]

# Define the LSTM model
input_sequence = Input(shape=(sequence_length, num_features))
#lstm_out = LSTM(units=64, return_sequences=True)(input_sequence)
lstm_out = LSTM(units=32, return_sequences=False, recurrent_initializer='glorot_uniform', implementation=2)(input_sequence)


# Define the attention mechanism
attention = Attention()([lstm_out, lstm_out])

# Concatenate the LSTM output and attention output
merged = Concatenate(axis=-1)([lstm_out, attention])

# Fully connected layer for final prediction
output = Dense(num_classes, activation='softmax')(merged)

# Create the hybrid model
model = Model(inputs=input_sequence, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(labels_for_training, axis=1)), y=np.argmax(labels_for_training, axis=1))
class_weights_dict = dict(enumerate(class_weights))

# Define early stopping and TensorBoard callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath='model/model_one_series_LSTM256_W_attention.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq='epoch',
    options=None  # Set options explicitly to None
)

# Train the model with class weights, early stopping, and TensorBoard
#model.fit(subset_X, subset_Y, epochs=10, batch_size=32, validation_data=(X_val, Y_val), class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard])
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_for_training, labels_for_training)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(32)

model.fit(train_dataset, epochs=30, validation_data=val_dataset, class_weight=class_weights_dict, callbacks=[early_stopping, tensorboard, checkpoint])


# Evaluate the model
loss, accuracy = model.evaluate(X_val, Y_val)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


In [ ]:
loaded_model = load_model('model/model_one_series_LSTM128_W_attention.h5')
class_labels = ["awake","wakeup","fallasleep","asleep"]

# Assuming you have trained your model and obtained predictions on the validation set
# Replace 'loaded_model' with the actual reference to your trained model

# Assuming X_val and Y_val are your validation data
print(Y_val[:10,:])

y_true = np.argmax(Y_val, axis=1)
y_pred = loaded_model.predict(X_val)
y_pred_class = np.argmax(y_pred, axis=1)

print(y_true[:10])
print(y_pred_class[:10])

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

loaded_model = load_model('model/model_one_series_LSTM128_W_attention.h5')
class_labels = ["awake","wakeup","fallasleep","asleep"]

# Assuming you have trained your model and obtained predictions on the validation set
# Replace 'loaded_model' with the actual reference to your trained model

# Assuming X_val and Y_val are your validation data
Y_val.head()

y_true = np.argmax(Y_val, axis=1)
y_pred = loaded_model.predict(X_val)
y_pred_class = np.argmax(y_pred, axis=1)

print(y_true[:10])
print(y_pred_class[:10])

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_class)

# Convert the confusion matrix to a DataFrame
confusion_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)

# Display the confusion matrix as a table
print("Confusion Matrix:")
print(confusion_df)

# Plot the confusion matrix as a heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

loaded_model = load_model('model/model_one_series_LSTM128_W_attention.h5')

# Assuming you have a new dataset 'new_X' and predicted classes 'predicted_classes'
# Replace 'loaded_model' with the actual reference to your trained model



# Example data (replace with your actual data)
X_train = three_d_array  # This is the windowed dataset the model was trained on.
window_size = three_d_array.shape[1]
prediction = loaded_model.predict(X_train)
predicted_classes = np.argmax(prediction, axis=1)



In [ ]:
prediction.shape

In [ ]:
Y_all.shape

In [ ]:
# Example list of row indices (replace with your actual indices)
row_indices = list(range(len(X_train)))

# Assuming 'original_time_series_length' is the length of the original time series
original_time_series_length = len(X_train)+50-1  # Replace with the actual length

# Initialize a list of zeros with the length of the original time series
result_list = [0] * original_time_series_length

# Iterate through the dataset and modify the result_list based on predicted classes
for idx, predicted_class in zip(row_indices, predicted_classes):
    if predicted_class == 1:  # 1 represents the 'wakeup' class
        #result_list[idx:idx + window_size] += 1
        for i in range(window_size):
            result_list[idx + i] += 1
    elif predicted_class == 2:  # 2 represents the 'fallasleep' class
        #result_list[idx:idx + window_size] += -1
        for i in range(window_size):
            result_list[idx + i] -= 1

# Now, 'result_list' contains the increments and decrements based on the predicted classes


In [ ]:
truth_list = [0] * original_time_series_length # a counterpart for result_list, considering the groundtruth labels

truth_classes = np.argmax(Y_all, axis=1)

# Iterate through the dataset and modify the truth_list based on predicted classes
for idx, truth_class in zip(row_indices, truth_classes):
    if truth_class == 1:  # 1 represents the 'wakeup' class
        #truth_list[idx:idx + window_size] += 1
        for i in range(window_size):
            truth_list[idx + i] += 1
    elif truth_class == 2:  # 2 represents the 'fallasleep' class
        #truth_list[idx:idx + window_size] -= 1
        for i in range(window_size):
            truth_list[idx + i] -= 1

In [ ]:
print(truth_classes[:10000])
print(truth_list[:10000])

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.plot(range(original_time_series_length)[:440000], truth_list[:440000], label='Truth List')
#plt.plot(range(original_time_series_length)[:100000], result_list[:100000], label='Predicted Result List')

# Adding labels and legend
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()

# Display the plot
plt.show()






In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 6))
# Plotting
plt.plot(range(original_time_series_length)[:440000], truth_list[:440000], label='Truth List')
plt.plot(range(original_time_series_length)[:440000], result_list[:440000], label='Predicted Result List', alpha=0.5)

# Adding labels and legend
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()

# Display the plot
plt.show()